In [0]:
!pip install pyspark

In [0]:
from pyspark.sql import SparkSession, DataFrameReader, SQLContext
from pyspark.context import SparkContext
sc = SparkContext()
spark = SparkSession(sc)
sqlContext = SQLContext(sc)

In [0]:
df = sqlContext.read.format('csv').options(header='true', inferSchema='true').load('daily_smoking_times.csv')

In [0]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorIndexer, MinMaxScaler
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, TrainValidationSplit
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import LogisticRegression

In [0]:
display(df)

DataFrame[Gender: string, Age: int, Time1: int, Time2: int, Time3: int, Time4: int, Time5: int, Time6: int, Time7: int]

In [0]:
df.printSchema()

NameError: ignored

In [0]:
from pyspark.sql.functions import col

In [0]:
df2 = df.select("Gender","Time1","Time2","Time3","Time4","Time5","Time6","Time7", col("Age").alias("label"))

In [0]:
df2

DataFrame[Gender: string, Time1: int, Time2: int, Time3: int, Time4: int, Time5: int, Time6: int, Time7: int, label: int]

In [0]:
splits = df2.randomSplit([0.7, 0.3])

In [0]:
train = splits[0]

In [0]:
test = splits[1].withColumnRenamed("label", "trueLabel")
train_rows = train.count()
test_rows = test.count()

In [0]:
print("Training Rows:", train_rows, " Testing Rows:", test_rows)
train.show(5)
test.show(5)

Training Rows: 4221  Testing Rows: 1779
+------+-----+-----+-----+-----+-----+-----+-----+-----+
|Gender|Time1|Time2|Time3|Time4|Time5|Time6|Time7|label|
+------+-----+-----+-----+-----+-----+-----+-----+-----+
|     F|    4|    8|   12|   14|   16|   20|    0|   19|
|     F|    4|    8|   12|   14|   16|   20|    3|   10|
|     F|    4|    8|   12|   14|   16|   21|    0|   41|
|     F|    4|    8|   12|   14|   16|   21|    2|   41|
|     F|    4|    8|   12|   14|   16|   22|    2|   36|
+------+-----+-----+-----+-----+-----+-----+-----+-----+
only showing top 5 rows

+------+-----+-----+-----+-----+-----+-----+-----+---------+
|Gender|Time1|Time2|Time3|Time4|Time5|Time6|Time7|trueLabel|
+------+-----+-----+-----+-----+-----+-----+-----+---------+
|     F|    4|    8|   12|   14|   16|   20|    3|       37|
|     F|    4|    8|   12|   14|   16|   21|    0|       12|
|     F|    4|    8|   12|   14|   16|   22|    1|       23|
|     F|    4|    8|   12|   14|   16|   23|    2|      

In [0]:
test.printSchema()

root
 |-- Gender: string (nullable = true)
 |-- Time1: integer (nullable = true)
 |-- Time2: integer (nullable = true)
 |-- Time3: integer (nullable = true)
 |-- Time4: integer (nullable = true)
 |-- Time5: integer (nullable = true)
 |-- Time6: integer (nullable = true)
 |-- Time7: integer (nullable = true)
 |-- trueLabel: integer (nullable = true)



In [0]:
strIdx = StringIndexer(inputCol = "label", outputCol = "typeAge")
labelIdx = StringIndexer(inputCol = "label", outputCol = "idxLabel")

In [0]:
catVect = VectorAssembler(inputCols = ["typeAge"], outputCol="ageFeatures")
catIdx = VectorIndexer(inputCol = catVect.getOutputCol(), outputCol = "idxAgeFeatures")
numVect = VectorAssembler(inputCols = ["Gender","Time1","Time2","Time3","Time4","Time5","Time6","Time7"], outputCol="numFeatures")

In [0]:
minMax = MinMaxScaler(inputCol = numVect.getOutputCol(), outputCol="normFeatures")
featVect = VectorAssembler(inputCols=["idxAgeFeatures", "normFeatures"], outputCol="features")

In [0]:
cl = []
pipeline = []

In [0]:
cl.insert(0, DecisionTreeClassifier(labelCol="idxLabel", featuresCol="features"))
cl.insert(1, RandomForestClassifier(labelCol="idxLabel", featuresCol="features"))
cl.insert(2, LogisticRegression(labelCol="idxLabel", featuresCol="features"))

In [0]:
for i in range(3):
    pipeline.insert(i, Pipeline(stages=[strIdx, labelIdx, catVect, catIdx, numVect, minMax, featVect, cl[i]]))
    #piplineModel = Pipeline.fit(train)
print("Pipeline complete!")


Pipeline complete!


In [0]:
model = []
paramGrid = (ParamGridBuilder().addGrid(cl[0].impurity, ("gini", "entropy")).addGrid(cl[0].maxDepth, [5, 10, 20]).addGrid(cl[0].maxBins, [5, 10, 20]).build())
cv = CrossValidator(estimator=pipeline[0], evaluator=BinaryClassificationEvaluator(), estimatorParamMaps=paramGrid, numFolds=5)

model.insert(0, cv.fit(train))


In [0]:
paramGrid2 = (ParamGridBuilder().addGrid(cl[1].impurity, ("gini", "entropy")).addGrid(cl[1].maxDepth, [5, 10, 20]).addGrid(cl[1].maxBins, [5, 10, 20]).build())
cv2 = CrossValidator(estimator=pipeline[1], evaluator=BinaryClassificationEvaluator(), estimatorParamMaps=paramGrid2, numFolds=5)

model.insert(1, cv2.fit(train))

In [0]:
paramGrid = (ParamGridBuilder().addGrid(cl[2].impurity, ("gini", "entropy")).addGrid(cl[2].maxDepth, [5, 10, 20]).addGrid(cl[2].maxBins, [5, 10, 20]).build())
cv = CrossValidator(estimator=pipeline[2], evaluator=BinaryClassificationEvaluator(), estimatorParamMaps=paramGrid, numFolds=5)

model.insert(2, cv3.fit(train))

In [0]:
prediction = [] 
predicted = []
for i in range(3):
  prediction.insert(i, model[i].transform(test))
  predicted.insert(i, prediction[i].select("features", "prediction", "probability", "trueLabel"))
  predicted[i].show(30)

In [0]:
#from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = BinaryClassificationEvaluator(
    labelCol="trueLabel", rawPredictionCol="prediction")
for i in range(3):
    #evaluator = MulticlassClassificationEvaluator(
    #labelCol="trueLabel", predictionCol="prediction", metricName="weightedRecall")
    areUPR = evaluator.evaluate(predicted[i], {evaluator.metricName: "areaUnderPR"})
    areUROC = evaluator.evaluate(predicted[i], {evaluator.metricName: "areaUnderROC"})
    print("AreaUnderPR = %g " % (areUPR))
    
    print("AreaUnderROC = %g " % (areUROC))

    tp = float(predicted[i].filter("prediction == 1.0 AND truelabel == 1").count())
    fp = float(predicted[i].filter("prediction == 1.0 AND truelabel == 0").count())
    tn = float(predicted[i].filter("prediction == 0.0 AND truelabel == 0").count())
    fn = float(predicted[i].filter("prediction == 0.0 AND truelabel == 1").count())

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    print("Precision = %g " % (precision))
    print("Recall = %g " % (recall))

    metrics = sqlContext.createDataFrame([
    ("TP", tp),
    ("FP", fp),
    ("TN", tn),
    ("FN", fn),
    ("Precision", tp / (tp + fp)),
    ("Recall", tp / (tp + fn))],["metric", "value"])
    metrics.show()